# Prediction on a Supernova
This notebook makes prediction of the element abundance and photosphere velocity and luminosity of a supernova, with a given spectrum and explosion time and a grid of density parameters.  


In [1]:
import os
import sys
import time
import tqdm
import glob
import yaml
import keras
import astropy
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K
import astropy.units as u
import astropy.constants as c
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from keras.models import Sequential,Model
from keras.callbacks import EarlyStopping
from dust_extinction.averages import GCC09_MWAvg
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.model_selection import train_test_split
from scipy.interpolate import interp1d,UnivariateSpline
from dust_extinction.parameter_averages import CCM89,F99
from keras.callbacks import TerminateOnNaN,EarlyStopping,ModelCheckpoint
from keras.layers import Dense,Dropout,LocallyConnected1D,AveragePooling1D,Flatten,Layer,Reshape
from keras.layers import Input,Conv1D,MaxPooling1D,BatchNormalization,Activation,Add,UpSampling1D,Concatenate

from GaussianNetworkKit import *

2023-07-01 14:46:09.756767: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-01 14:46:10.638722: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_15503/3143503893.py:20: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


# The Observation
A list of the supernovae included in the paper are stored in the file "dm15WithDens.csv". You can take a look at this list and see which supernova is interesting. 

In [2]:
wave=np.genfromtxt('Prim.ascii')[:,0]
dm15List=pd.read_csv('dm15EbvFinal.csv',index_col=0)
#snIndexer=int(sys.argv[1])

def Normalizer(spec,shortwave=6500,longwave=7500):
    small=np.argmin(abs(spec[:,0]-shortwave))
    long=np.argmin(abs(spec[:,0]-longwave))
    if small<long:spec[:,1]=spec[:,1]/np.average(spec[small:long,1])
    if small>long:spec[:,1]=spec[:,1]/np.average(spec[long:small,1])
    return spec
def windowSpec(spec):
    spFunc=interp1d(spec[:,0],spec[:,1],fill_value=np.nan,bounds_error=False)
    smFlux=spFunc(wave)
    smFlux=smFlux/np.nanmean(smFlux)
    smFlux[np.isnan(smFlux)]=-1
    return np.array([wave,smFlux]).T
def specListMaker(starTableDir,redshift,extMW=0,extHost=0,telNameGetter=False):
    starTable=pd.read_csv(starTableDir+'starTableDen.csv')#'../ObserveSpectra/GoodSpec/SN2011fe/starTable.csv'
    X_snemo=[]
    timeCollect=[]
    telNameList=[]
    for i in range(len(starTable)):
        if starTable['Phase'][i]>20 or starTable['Phase'][i]<-10:continue
        fileName=starTable['Ascii file'].iloc[i]
        fileName=glob.glob(starTableDir+fileName.split('.dat')[0]+'*')[0]
        spec=np.genfromtxt(fileName)
        spec[:,1]=spec[:,1]/GCC09_MWAvg().extinguish(spec[:,0]*u.Angstrom,Ebv=extMW)
        spec[:,0]=spec[:,0]/(1+redshift)
        spec[:,1]=spec[:,1]/F99().extinguish(spec[:,0]*u.Angstrom,Ebv=extHost)
        spec=Normalizer(spec)
        spResa=windowSpec(spec)
        if np.max(spResa[:,1])>20:continue
        X_snemo.append(spResa[:,1].reshape([2000,1]))
        timeCollect.append(starTable['Phase'][i])
        if telNameGetter:
            telNameList.append(starTable['Tel'][i])
    X_snemo=np.array(X_snemo)
    timeCollect=np.array(timeCollect)+19
    if telNameGetter:
        return X_snemo,timeCollect,telNameList
    else:
        return X_snemo,timeCollect


# The Training Data Set
Reads the training and testing data set into the program, but not really used. The "YauxNorm" will be used to reconstruct the real luminosity and explosion time and photosphere velocity from neural network predictions.  

In [3]:
X_train=np.load('../4_Fit/snakeHead/X_train.npy')[:200]
X_test=np.load('../4_Fit/snakeHead/X_test.npy')[:200]
Y_train=np.load('../4_Fit/snakeHead/Y_train.npy')[:200]
Y_test=np.load('../4_Fit/snakeHead/Y_test.npy')[:200]
Yaux_train=np.load('../4_Fit/snakeHead/Yaux_train.npy')[:200]
Yaux_test=np.load('../4_Fit/snakeHead/Yaux_test.npy')[:200]
YauxNorm=np.load('../4_Fit/snakeHead/YauxNorm.npy')

In [4]:
def MRNNSoftMaxESMake(CellNumber=7,outter=0,X_train=X_train,X_test=X_test,Y_train=Y_train,Y_test=Y_test,Yaux_train=Yaux_train,Yaux_test=Yaux_test):
    INput=Input(shape=(X_train.shape[1],1,))
    conv1=Conv1D(8,15,strides=2,padding='same')(INput)
    conv1=Conv1D(16,3,strides=1,padding='same')(conv1)
    batc1=BatchNormalization()(conv1)
    acti1=Activation('selu')(batc1)
    pool1=MaxPooling1D(2)(acti1)
    
    conv2=Conv1D(8,1)(pool1)
    batc2=BatchNormalization()(conv2)
    acti2=Activation('selu')(batc2)
    conv3=Conv1D(16,3,padding='same')(acti2)
    
    adds=[pool1]
    
    addi=Add()(adds+[conv3])
    adds.append(addi)
    
    for i in range(CellNumber):
        conv2=Conv1D(8,1)(addi)
        batc2=BatchNormalization()(conv2)
        acti2=Activation('selu')(batc2)
        conv3=Conv1D(16,3,padding='same')(acti2)
        addi=Add()(adds+[conv3])
        adds.append(addi)
    
    batc2=BatchNormalization()(addi)
    
    flat1=keras.layers.Flatten()(batc2)
    drop1=Dropout(0.2)(flat1)
    dens1=Dense(256,activation='selu')(drop1)
    
    INput2=Input(shape=(3,))
    dens2=Dense(6,activation='selu')(INput2)
    dens2=Dense(9,activation='selu')(dens2)
    conc1=Concatenate()([INput2,dens2])
    dens2=Dense(21,activation='selu')(conc1)
    conc1=Concatenate()([INput2,dens2])
    dens2=Dense(45,activation='selu')(conc1)
    conc1=Concatenate()([INput2,dens2])
    
    conc2=Concatenate()([conc1,dens1])
    dens3=Dense(384,activation='selu')(conc2)
    drop2=Dropout(0.2)(dens3)
    
    dens3=Dense(256,activation='selu')(drop2)
    dens3=Dense(256,activation='selu')(dens3)
    dens3=Dense(256,activation='selu')(dens3)
    dens3=Dense(256,activation='selu')(dens3)
    
    if outter==6:mu,sig=GaussianLayer(2,name='ld_out')(dens3)
    else:mu,sig=GaussianLayer(30,hardMax=False,name='zone_'+str(outter)+'_out')(dens3)
    model=Model(inputs=[INput,INput2],outputs=mu)
    print(model.summary())
    opt=keras.optimizers.Adam(lr=0.00003,decay=1e-6)
    model.compile(optimizer=opt,loss=custom_loss(sig))
    return model


# The Neural Network
The neural network parameters are read here. I refer to the neural network loss function metrics to find the best performance neural networks. There are 7 neural networks in total, 6 for element abundances, the other is for the luminosity, etc.  

In [9]:
intermediateModels=[]
for outter in range(7):
    valiLossList=[]
    for mdInd in range(10):
        try:
            his2=pd.read_csv('../2_Train/MdSaver/110KLogML/ES_'+str(outter)+'_'+str(mdInd)+'_His2.csv')
            valiLoss=his2['val_loss'].iloc[-1]
            if np.isnan(valiLoss):valiLoss=np.inf
        except:valiLoss=np.inf
        valiLossList.append(valiLoss)
    mdIndChos=np.argmin(valiLossList)
    model=MRNNSoftMaxESMake(outter=outter)
    model.load_weights('../2_Train/MdSaver/110KLogML/Model_'+str(outter)+'_'+str(mdIndChos)+'.hdf')
    #model.load_weights('/home/gesa/SuperCode/GithubUpload/2_Train/MdSaver/110KLogML/Model_'+str(outter)+'_'+str(mdIndChos)+'.hdf')
    if outter==6:
        outLayerName='ld_out'
    else:outLayerName='zone_'+str(outter)+'_out'
    intermediateModels.append(K.function(inputs=[model.input[0],model.input[1]],outputs=model.get_layer(outLayerName).output))


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 2000, 1)]    0           []                               
                                                                                                  
 conv1d_18 (Conv1D)             (None, 1000, 8)      128         ['input_3[0][0]']                
                                                                                                  
 conv1d_19 (Conv1D)             (None, 1000, 16)     400         ['conv1d_18[0][0]']              
                                                                                                  
 batch_normalization_10 (BatchN  (None, 1000, 16)    64          ['conv1d_19[0][0]']              
 ormalization)                                                                              

                                                                  'add_11[0][0]',                 
                                                                  'conv1d_29[0][0]']              
                                                                                                  
 conv1d_30 (Conv1D)             (None, 500, 8)       136         ['add_12[0][0]']                 
                                                                                                  
 batch_normalization_16 (BatchN  (None, 500, 8)      32          ['conv1d_30[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_15 (Activation)     (None, 500, 8)       0           ['batch_normalization_16[0][0]'] 
                                                                                                  
 conv1d_31

 dense_10 (Dense)               (None, 256)          2048256     ['dropout_2[0][0]']              
                                                                                                  
 concatenate_7 (Concatenate)    (None, 304)          0           ['concatenate_6[0][0]',          
                                                                  'dense_10[0][0]']               
                                                                                                  
 dense_15 (Dense)               (None, 384)          117120      ['concatenate_7[0][0]']          
                                                                                                  
 dropout_3 (Dropout)            (None, 384)          0           ['dense_15[0][0]']               
                                                                                                  
 dense_16 (Dense)               (None, 256)          98560       ['dropout_3[0][0]']              
          

/home/gesa/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2023-07-01 14:51:46.067315: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open /home/gesa/SuperCode/GithubUpload/2_Train/MdSaver/110KLogML/Model_0_1.hdf: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 2000, 1)]    0           []                               
                                                                                                  
 conv1d_36 (Conv1D)             (None, 1000, 8)      128         ['input_5[0][0]']                
                                                                                                  
 conv1d_37 (Conv1D)             (None, 1000, 16)     400         ['conv1d_36[0][0]']              
                                                                                                  
 batch_normalization_20 (BatchN  (None, 1000, 16)    64          ['conv1d_37[0][0]']              
 ormalization)                                                                              

...

/home/gesa/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2023-07-01 14:51:50.602232: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open /home/gesa/SuperCode/GithubUpload/2_Train/MdSaver/110KLogML/Model_5_0.hdf: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 2000, 1)]    0           []                               
                                                                                                  
 conv1d_126 (Conv1D)            (None, 1000, 8)      128         ['input_15[0][0]']               
                                                                                                  
 conv1d_127 (Conv1D)            (None, 1000, 16)     400         ['conv1d_126[0][0]']             
                                                                                                  
 batch_normalization_70 (BatchN  (None, 1000, 16)    64          ['conv1d_127[0][0]']             
 ormalization)                                                                             

                                                                  'add_59[0][0]',                 
                                                                  'conv1d_137[0][0]']             
                                                                                                  
 conv1d_138 (Conv1D)            (None, 500, 8)       136         ['add_60[0][0]']                 
                                                                                                  
 batch_normalization_76 (BatchN  (None, 500, 8)      32          ['conv1d_138[0][0]']             
 ormalization)                                                                                    
                                                                                                  
 activation_69 (Activation)     (None, 500, 8)       0           ['batch_normalization_76[0][0]'] 
                                                                                                  
 conv1d_13

 dense_70 (Dense)               (None, 256)          2048256     ['dropout_14[0][0]']             
                                                                                                  
 concatenate_31 (Concatenate)   (None, 304)          0           ['concatenate_30[0][0]',         
                                                                  'dense_70[0][0]']               
                                                                                                  
 dense_75 (Dense)               (None, 384)          117120      ['concatenate_31[0][0]']         
                                                                                                  
 dropout_15 (Dropout)           (None, 384)          0           ['dense_75[0][0]']               
                                                                                                  
 dense_76 (Dense)               (None, 256)          98560       ['dropout_15[0][0]']             
          

/home/gesa/anaconda3/envs/tensorflow/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
2023-07-01 14:51:51.471360: W tensorflow/core/util/tensor_slice_reader.cc:97] Could not open /home/gesa/SuperCode/GithubUpload/2_Train/MdSaver/110KLogML/Model_6_2.hdf: DATA_LOSS: not an sstable (bad magic number): perhaps your file is in a different file format and you need to use a different restore operator?


In [10]:
def ModelPredictor(modelList,specList,auxInList):
    matExport=np.zeros([len(specList),6,30])
    errExport=np.zeros([len(specList),6,30])
    for j in range(6):
        mu,sigma=modelList[j]([specList,auxInList])
        matExport[:,j,:]=mu
        errExport[:,j,:]=sigma**0.5
    j=6
    mu,sigma=modelList[j]([specList,auxInList])
    sigma=sigma**0.5
    mu[:,0]=mu[:,0]*YauxNorm[1,0]+YauxNorm[0,0]
    mu[:,1]=mu[:,1]*YauxNorm[1,2]+YauxNorm[0,2]
    sigma[:,0]=sigma[:,0]*YauxNorm[1,0]
    sigma[:,1]=sigma[:,1]*YauxNorm[1,2]
    return matExport,errExport,mu,sigma

# The Spectra
Here, the program reads all the spectra into the notebook referring to the data stored in "ObserveSpec4Wash" folder.  
You can take a look at variable "snNameList" to see which of the supernovae are included.  
Not all the supernovae in the "dm15EbvFinal.csv" file are included, some may not have spectra between 10 days and 40 days.  

In [11]:
XspecList,timeList,snNameList,dm15ChosL,dm15ErrL=[],[],[],[],[]
for i in range(len(dm15List)):
    snName=dm15List['ObjName'].iloc[i]
    redshift=dm15List['Redshift'].iloc[i]
    if os.path.exists('ObserveSpec4Wash/'+snName+'/starTableDen.csv')==False:continue
    X_snemo,timeCollect=specListMaker('ObserveSpec4Wash/'+snName+'/',redshift,extHost=dm15List['EBVhost'].iloc[i],extMW=dm15List['EBVmw'].iloc[i])
    if len(X_snemo)==0:continue
    timeList.append(timeCollect)
    dm15ChosL.append(dm15List['DM15'].iloc[i])
    dm15ErrL.append(dm15List['DM15err'].iloc[i])
    snNameList.append(dm15List['ObjName'].iloc[i])
    XspecList.append(X_snemo)

# Pick a Supernova
Here you can pick a supernova, like number 10, to make predictions.  
The density parameter grid is also made at here.  

In [12]:
snIndexer=10
i=snIndexer
densPivotRange=np.arange(0.2,2.01,0.1)
densSlopeRange=np.arange(0,2.01,0.1)

# Predict
Here it is the real prediction code. The neural networks will make predictions of the element and auxiliary data and their error bars, and store them into "matBig.npy" (element data), "auxBig.npy" (luminosity, etc), "errBig.npy" (element data error), "aerBig.npy" (luminosity, etc, error). Also, the spectra resampled to the neural network input format is also stored, in the "Xinput.npy" file.  

In [15]:
os.popen('mkdir -p predOutMassive/'+snNameList[i])
X_snemo=XspecList[i]
timeCollect=timeList[i]
matBig=np.zeros([len(densPivotRange),len(densSlopeRange),len(X_snemo),6,30])
errBig=np.zeros([len(densPivotRange),len(densSlopeRange),len(X_snemo),6,30])
auxBig=np.zeros([len(densPivotRange),len(densSlopeRange),len(X_snemo),5])
aerBig=np.zeros([len(densPivotRange),len(densSlopeRange),len(X_snemo),5])
for densNInd,densN in enumerate(densPivotRange):
    for densDInd,densD in enumerate(densSlopeRange):
        densN=float(round(densN,3))
        densD=float(round(densD,3))
        timeNorm=(timeCollect-YauxNorm[0,1])/YauxNorm[1,1]
        timeNorm=timeNorm.reshape([-1,1])
        dens1Norm=(np.ones([len(X_snemo),1])*densN-YauxNorm[0,3])/YauxNorm[1,3]
        dens2Norm=(np.ones([len(X_snemo),1])*densD-YauxNorm[0,4])/YauxNorm[1,4]
        auxIn=np.hstack([timeNorm,dens1Norm,dens2Norm])
        matExport,errExport,mu,sigma=ModelPredictor(intermediateModels,X_snemo,auxIn)

        auxList=np.zeros([len(X_snemo),5])
        auxList[:,0]=mu[:,0]
        auxList[:,2]=mu[:,1]
        auxList[:,1]=timeCollect
        auxList[:,3]=densN
        auxList[:,4]=densD
        sigList=np.zeros([len(X_snemo),5])
        sigList[:,0]=sigma[:,0]
        sigList[:,2]=sigma[:,1]
        matBig[densNInd,densDInd]=matExport
        errBig[densNInd,densDInd]=errExport
        auxBig[densNInd,densDInd]=auxList
        aerBig[densNInd,densDInd]=sigList
matBig=10**matBig
np.save('predOutMassive/'+snNameList[i]+'/matBig.npy',matBig)
np.save('predOutMassive/'+snNameList[i]+'/errBig.npy',errBig)
np.save('predOutMassive/'+snNameList[i]+'/auxBig.npy',auxBig)
np.save('predOutMassive/'+snNameList[i]+'/aerBig.npy',aerBig)
np.save('predOutMassive/'+snNameList[i]+'/Xinput.npy',X_snemo)

In [14]:
snNameList

['SN2000cw',
 'SN2006gr',
 'SN1999dq',
 'SN2004gs',
 'SN2001cp',
 'SN2008bq',
 'SN1998dh',
 'SN2006N',
 'SN2001en',
 'SN2001cj',
 'SN2007cq',
 'SN2006gj',
 'SN1999ee',
 'SN2007ap',
 'SN2006D',
 'SN2003ch',
 'SN1998ec',
 'SN2007bc',
 'SN2001el',
 'SN1995al',
 'SN1997bq',
 'SN2000cn',
 'SN2002kf',
 'SN2001da',
 'SN2007bd',
 'SN2006cc',
 'SN1991S',
 'SN2007if',
 'SN2010ev',
 'SN1994S',
 'SN2015F',
 'SN2006bq',
 'SN2012cg',
 'SN2007ci',
 'SN1991M',
 'SN2006or',
 'SN2009dc',
 'SN2007F',
 'SN1989B',
 'SN2009ig',
 'SN2004gu',
 'SN2009Y',
 'SN1994Q',
 'SN2006fw',
 'SN2003D',
 'SN1999ac',
 'SN2006et',
 'SN2002hd',
 'SN2002hu',
 'SN2011fe',
 'SN2004fu',
 'SN2013dy',
 'SN2006cz',
 'SN2009F',
 'SN2007ca',
 'SN2002cf',
 'SN2002hw',
 'SN2006cf',
 'SN2006bt',
 'SN2004fz',
 'SN1996X',
 'SN1992A',
 'SN2007as',
 'SN2005M',
 'SN2006sr',
 'SN2003cq',
 'SN2007sr',
 'SN2001ex',
 'SN2002cr',
 'SN2002dj',
 'SN1998es',
 'SN2006S',
 'SN2005eq',
 'SN2003W',
 'SN2008bc',
 'SN2006bw',
 'SN2007hj',
 'SN2006gz',
 'S